# Exploring UCC cluster data

This notebook allows you to explore the data for each cluster listed in the UCC. The UCC cluster datafiles contain the frame processed by the [fastMP](https://asteca.readthedocs.io/en/latest/apidocs/asteca/asteca.membership.html#asteca.membership.Membership.fastmp) membership algorithm.

Alternatively, you can also access the datafiles for the [Hunt & Reffert (2023)](https://ui.adsabs.harvard.edu/abs/2023A%26A...673A.114H) (HUNT23) and [Cantat-Gaudin et al. (2020)](https://ui.adsabs.harvard.edu/abs/2020A%26A...640A...1C) (CANTAT20) articles if available for a given cluster.

## Import packages and load data

First we **define the name of the cluster to be analyzed**. In this example we select the cluster `Blanco 1` (it doesn't matter if you use upper or lower case):

In [ ]:
cluster = "blanco 1"

We then import the required packages:

In [ ]:
import pandas as pd
import numpy as np
from urllib.error import HTTPError
from bokeh.plotting import figure, show
from bokeh.models import LinearColorMapper
from bokeh.models import ColumnDataSource
from bokeh.models import ColorBar
from bokeh.models import Range1d
from bokeh.io import output_notebook
output_notebook()

The next step is to load the cluster data into a `pandas` dataframe. To do this we first need to define the function that handles the loading.


In [ ]:
def load_cluster(cluster, db_id):
    """Function that handles loading a cluster's datafile in any of the
    UCC, HUNT23 or CANTAT20 databases, from the UCC repositories."""
    # Proper file name format
    file_name = cluster.lower().replace('_', '').replace(' ', '').replace('-', '')
    # Proper databse name format
    _db_id = '_'+db_id if db_id != 'UCC' else ''
    # Check all repositories where datafiles are stored
    for qfolder in ('1N', '1P', '2N', '2P', '3N', '3P', '4N', '4P'):
        try:
            repo_root = f"https://github.com/ucc23/Q{qfolder}/raw/main/datafiles/"
            path = repo_root + file_name + f"{_db_id}.parquet"
            df = pd.read_parquet(path)
            print(f"Cluster '{cluster}' loaded from '{db_id}' database")
            return df
        except HTTPError:
            pass
    raise ValueError(f"Cluster '{cluster}' not found in {db_id} database")

We can now load a cluster with data from the UCC, HUNT23 or CANTAT20, assuming that cluster was processed in either of those articles. To do this select either `UCC`, `HUNT23` or `CANTAT20` in the `db_id` variable below and load.

In [ ]:
db_id = 'UCC'
# db_id = 'HUNT23'
# db_id = 'CANTAT20'

# Load the dataframe
df = load_cluster(cluster, db_id)

The most probable members in the UCC database are stored using a probability cut of `P>0.5`. A minimum number of member stars is set to `25`, so that if less than `25` stars have `P>0.5` then the most probable members are those `25` stars with the largest probability values below 0.5. Other databases handle this differently.

## Define plotting functions


Define functions to generate scatter plots and histograms:

In [ ]:
def scatter_plot(x, y, col, flip_yaxis=False):
    members = ColumnDataSource({'xm':df[x], 'ym':df[y], 'col':df[col]})
    cmap = LinearColorMapper(palette="Plasma256", low = df[col].min(), high = 1)

    p = figure()
    p.scatter("xm", "ym", size=10, source=members, line_color='black', alpha=.75,
              fill_color={"field":"col", "transform":cmap})
    if flip_yaxis:
        p.y_range.flipped = True
    bar = ColorBar(color_mapper=cmap)
    p.add_layout(bar, "right")
    p.xaxis.axis_label = x
    p.yaxis.axis_label = y
    show(p)


def histo_plot(x):
    p = figure()
    # Histogram for member stars
    hist, edges = np.histogram(df[x], bins=20)
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
            fill_color="skyblue", line_color="white", alpha=.75)
    # Vertical line
    p.ray(x=[np.nanmedian(df[x])], y=[0], length=0, angle=90,
          angle_units='deg', line_width=3, line_color='red')
    left, right = np.nanmin(df[x]) * 0.9, np.nanmax(df[x]) * 1.1
    p.x_range=Range1d(left, right)
    p.xaxis.axis_label = x
    p.yaxis.axis_label = "N"
    show(p)

## Generate interactive plots

Now we can generate some interactive plots. For example the distribution of galactic coordinates for member stars, colored by their membership probability

In [ ]:
x, y, col = 'GLON', 'GLAT', 'probs'
scatter_plot(x, y, col)

And a color-magnitude diagram

In [ ]:
x, y, col = 'BP-RP', 'Gmag', 'probs'
scatter_plot(x, y, col, True)

Distribution of proper motions

In [ ]:
x, y, col = 'pmRA', 'pmDE', 'probs'
scatter_plot(x, y, col)

Histogram of the parallaxes with the median value of the selected members shown as a red vertical line

In [ ]:
histo_plot('Plx')